<a href="https://colab.research.google.com/github/Amitparikh1/CDC-Dash-Workshop/blob/main/Interactive_Data_Apps_in_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Interactive Data Apps in Dash**

Github Link: https://github.com/Amitparikh1/CDC-Dash-Workshop

## Attendance Form

https://forms.gle/htBzaYRSUs23kgLBA

## Introduction

### What is Dash?
Dash is a Python framework created by Plotly that makes it fast and easy to create interactive web applications.

It's built on top of: 


*   Flask
*   Plotly
*   React

Don't need to know HTML, CSS (kind-of), Javascript. **Only Python!**

### Why use Dash?
Just because Dash is easy, doesn't mean it's not powerful. Dash is used to create production grade applications by many businesses including Tesla, S&P Global, Cisco and more. 
[Cool uses of Dash.](https://medium.com/plotly/7-new-dash-apps-made-by-the-dash-community-196998112ce3)









## Getting Started

There are two main parts of a Dash App:


1.   Layout
2.   Callback

### Layout
The Layout defines how the app looks and how it can be interacted with. It basically says what elements will be on the page (graphs, dropdowns, headers etc.)
There are two types of Layout components: 

1.   [**Dash HTML Components**](https://dash.plotly.com/dash-html-components) - allow us to create HTML components like headings, images, paragraphs etc. in Python
2.   [**Dash Core Components** ](https://dash.plotly.com/dash-core-components)- allow us to create graphs, dropdowns, sliders, etc. 

### Callback
A function that allows us to make components interactive.
Callback function takes in Input, Output, and/or State arguments.


*   Input 
    *  Defines components who will trigger the callback function to fire when changed.
    * component_id: The id of the input component in the Layout
    * component_property: The property in the input component that we are watching for. 
*   State
    *  Defines components that read input from the Layout but won't fire the callback function when changed.
    * Takes in component_id and component_property like Input
*   Output
    *  Defines components in the Layout that will be updated when the callback function is returned.
    * Takes in component_id and component_property like Input








### Import Packages

In [ ]:
# Using Jupyter Dash because we're running out of a Colab notebook
!pip install -q jupyter-dash
# Imports
import pandas as pd
import plotly
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
# Jupyter Dash Imports
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

### Read in Data

In [2]:
# Read in a data set to dispaly in our interactive app
df_master = pd.read_csv('https://raw.githubusercontent.com/dhavalpotdar/Gapminder-World-Data-Analysis/master/data/master.csv')
# Check for nan values
print(df_master.info())
# Get rid of columns with nan values
df = df_master[['country', 'year', 'gdp_pc', 'life_exp_yrs', 'youth_empl_rate', 'pop_density']]
# Dictionary of col names to readable names 
col_names = {
    'gdp_pc': 'GDP Per Capita', 
    'life_exp_yrs': 'Life Expectancy (years)', 
    'youth_empl_rate': 'Youth Employment Rate', 
    'pop_density': 'Population Density'
}

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066 entries, 0 to 3065
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country               3066 non-null   object 
 1   year                  3066 non-null   int64  
 2   literacy_rate         443 non-null    float64
 3   gdp_pc                3066 non-null   int64  
 4   life_exp_yrs          3066 non-null   float64
 5   youth_empl_rate       3066 non-null   float64
 6   percent_forest_cover  3012 non-null   float64
 7   hdi                   2758 non-null   float64
 8   pop_density           3066 non-null   float64
dtypes: float64(6), int64(2), object(1)
memory usage: 215.7+ KB
None


### Create a Static Page

In [ ]:
# Create a base plot to display without any interactivity
# The plot will be a timeseries of the life expectancy in the US 
df_base = df[df['country'] == "India"]
fig = px.line(data_frame=df_base, x='year', y='life_exp_yrs', title="Average Life Expectancy in India (1991-2011)")
fig

In [ ]:
# Build App
app = JupyterDash(__name__)

# App Layout
app.layout = html.Div([
    # A Dash HTML Component
    html.H1("Welcome to CDC!", style={'text-align':'center'}), 
    # A Dash Core Component
    dcc.Graph(figure=fig)
    ])

# Run app
app.run_server(mode='external', port='8070')

### Make Our Page Interactive

In [ ]:
# What data do we want to display interactively?
df.head()

In [ ]:
# Build App
app = JupyterDash(__name__)

# App Layout 
app.layout = html.Div([
    # A Dash HTML Component
    html.H1("Welcome to CDC!", style={'text-align':'center'}), 
    # Dash Core Component
    dcc.Graph(id="timeseries_plot"),
    html.Label(
        ["Select a metric to show:",
        # Dropdown to specify metric 
        dcc.Dropdown(
            # ID to reference component by
            id="metric_selector",
            # Options in our dropdown, we'll use our col_names dictionary from above
            options=[{'label': col_names[col], 'value':col} for col in col_names.keys()], 
            # Default value to start at
            value='life_exp_yrs',
        )   
    ]),
    # Label for multi-select dropdown
    html.Label(
        ["Select countries to display: ", 
        # Multi-select Dropdown to select countries to display
        dcc.Dropdown(
            # ID to reference component by
            id="country_selector",
            # Multi-select Dropdown
            multi=True, 
            # Set options to a dictionary of every country in the dataset 
            options=[{"label": country, "value": country} for country in df['country'].unique()], 
            # Default Value
            value=["India"], 
        )       
    ])
    ])

# Define callback to update graph
@app.callback(
    Output(component_id="timeseries_plot", component_property="figure"),
    [Input(component_id="metric_selector", component_property="value"), Input(component_id="country_selector", component_property="value")]
)
def update_figure(selected_metric, selected_countries):
  print(selected_metric)
  # Filter data frame to only selected countries
  df_filtered = df[df['country'].isin(selected_countries)]
  # Create graph with filtered df and specified metric
  fig = px.line(data_frame=df_filtered, x='year', y=selected_metric, color="country", title=f'{col_names[selected_metric]} in selected countries (1991-2011)')
  # Return figure to the callback functions specified Output
  return fig

# Run app
app.run_server(mode='external', port='8071', debug=False)

### CHALLENGE: Add a RangeSlider to Change Range of Years

**Starter Code to Try on Your Own**


In [ ]:
# Build App
app = JupyterDash(__name__)

# App Layout 
app.layout = html.Div([
    # A Dash HTML Component
    html.H1("Welcome to CDC!", style={'text-align':'center'}), 
    # Dash Core Component
    dcc.Graph(id="timeseries_plot"),
    html.Label(
        ["Select a metric to show:",
        # Dropdown to specify metric 
        dcc.Dropdown(
            # ID to reference component by
            id="metric_selector",
            # Options in our dropdown, we'll use our col_names dictionary from above
            options=[{'label': col_names[col], 'value':col} for col in col_names.keys()], 
            # Default value to start at
            value='life_exp_yrs',
        )   
    ]),
    # Label for multi-select dropdown
    html.Label(
        ["Select countries to display: ", 
        # Multi-select Dropdown to select countries to display
        dcc.Dropdown(
            # ID to reference component by
            id="country_selector",
            # Multi-select Dropdown
            multi=True, 
            # Set options to a dictionary of every country in the dataset 
            options=[{"label": country, "value": country} for country in df['country'].unique()], 
            # Default Value
            value=["India"], 
        )       
    ])
    ])

# Define callback to update graph
@app.callback(
    Output(component_id="timeseries_plot", component_property="figure"),
    [Input(component_id="metric_selector", component_property="value"), Input(component_id="country_selector", component_property="value")]
)
def update_figure(selected_metric, selected_countries):
  # Filter data frame to only selected countries
  df_filtered = df[df['country'].isin(selected_countries)]
  # Create graph with filtered df and specified metric
  fig = px.line(data_frame=df_filtered, x='year', y=selected_metric, color="country", title=f'{col_names[selected_metric]} in selected countries (1991-2011)')
  # Return figure to the callback functions specified Output
  return fig

# Run app
app.run_server(mode='external', port='8072')

**Sample Solution**

In [ ]:
#@title Sample RangeSlider Solution
# Build App
app = JupyterDash(__name__)

# App Layout 
app.layout = html.Div([
    # A Dash HTML Component
    html.H1("Welcome to CDC!", style={'text-align':'center'}), 
    # Dash Core Component
    dcc.Graph(id="timeseries_plot"),
    html.Label(
        ["Select a metric to show:",
        # Dropdown to specify metric 
        dcc.Dropdown(
            # ID to reference component by
            id="metric_selector",
            # Options in our dropdown, we'll use our col_names dictionary from above
            options=[{'label': col_names[col], 'value':col} for col in col_names.keys()], 
            # Default value to start at
            value='life_exp_yrs',
        )   
    ]),
    # Label for multi-select dropdown
    html.Label(
        ["Select countries to display: ", 
        # Multi-select Dropdown to select countries to display
        dcc.Dropdown(
            # ID to reference component by
            id="country_selector",
            # Multi-select Dropdown
            multi=True, 
            # Set options to a dictionary of every country in the dataset 
            options=[{"label": country, "value": country} for country in df['country'].unique()], 
            # Default Value
            value=["India"], 
        )       
    ]),
    # Label for slider
    html.Label(
        ["Choose a range of years to display:", 
        # RangeSlider to specify range of years to display
        dcc.RangeSlider(
          id='year_slider',
          min=df['year'].min(),
          max=df['year'].max(),
          step=1,
          value=[df['year'].min(), df['year'].max()],
          marks={x:str(x) for x in range(df['year'].min(), df['year'].max() + 1)}
      )
        ]
    )
    ])

# Define callback to update graph
@app.callback(
    Output(component_id="timeseries_plot", component_property="figure"),
    [Input(component_id="metric_selector", component_property="value"), Input(component_id="country_selector", component_property="value"), Input(component_id="year_slider", component_property="value")]
)
def update_figure(selected_metric, selected_countries, year_range):
  # Get min and max year from year_range
  min_year = min(year_range)
  max_year = max(year_range)
  # Filter data frame to year range
  df_filtered = df[(df['year'] >= min_year) & (df['year'] <= max_year)]
  # Filter data frame to only selected countries
  df_filtered = df_filtered[df_filtered['country'].isin(selected_countries)]
  # Create graph with filtered df and specified metric
  fig = px.line(data_frame=df_filtered, x='year', y=selected_metric, color="country", title=f'{col_names[selected_metric]} in selected countries (1991-2011)')
  # Return figure to the callback functions specified Output
  return fig

# Run app
app.run_server(mode='external', port='8073')